# 提示词工程

In [1]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

# 读取城市数据

In [2]:
# 城市数据
# 110000 北京市
# 110100 北京
# 110101 东城区
with open('data/city.txt','r',encoding='utf-8') as fp:
    # 读取文件的所有行，并将每一行作为一个字符串存储到列表city_list中。
    # 处理后city_list的内容将是['110000 北京市\n', '110100 北京\n', '110101 东城区\n']
    city_list=fp.readlines()
    # 使用列表推导式对city_list进行处理，提取出每行的第二个字段。
    # strip()方法移除字符串两端的空白字符（包括换行符\n和空格）。
    # split(' ')将字符串按空格分割成一个列表。
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [3]:
city_list

['北京市',
 '北京',
 '东城区',
 '西城区',
 '朝阳区',
 '丰台区',
 '石景山区',
 '海淀区',
 '门头沟区',
 '房山区',
 '通州区',
 '顺义区',
 '昌平区',
 '大兴区',
 '怀柔区',
 '平谷区',
 '密云区',
 '延庆区',
 '天津市',
 '天津',
 '和平区',
 '河东区',
 '河西区',
 '南开区',
 '河北区',
 '红桥区',
 '东丽区',
 '西青区',
 '津南区',
 '北辰区',
 '武清区',
 '宝坻区',
 '滨海新区',
 '宁河区',
 '静海区',
 '蓟州区',
 '河北省',
 '石家庄市',
 '市辖区',
 '长安区',
 '桥西区',
 '新华区',
 '井陉矿区',
 '裕华区',
 '藁城区',
 '鹿泉区',
 '栾城区',
 '井陉县',
 '正定县',
 '行唐县',
 '灵寿县',
 '高邑县',
 '深泽县',
 '赞皇县',
 '无极县',
 '平山县',
 '元氏县',
 '赵县',
 '晋州市',
 '新乐市',
 '唐山市',
 '市辖区',
 '路南区',
 '路北区',
 '古冶区',
 '开平区',
 '丰南区',
 '丰润区',
 '曹妃甸区',
 '滦县',
 '滦南县',
 '乐亭县',
 '迁西县',
 '玉田县',
 '遵化市',
 '迁安市',
 '秦皇岛市',
 '市辖区',
 '海港区',
 '山海关区',
 '北戴河区',
 '抚宁区',
 '青龙满族自治县',
 '昌黎县',
 '卢龙县',
 '邯郸市',
 '市辖区',
 '邯山区',
 '丛台区',
 '复兴区',
 '峰峰矿区',
 '邯郸县',
 '临漳县',
 '成安县',
 '大名县',
 '涉县',
 '磁县',
 '肥乡县',
 '永年县',
 '邱县',
 '鸡泽县',
 '广平县',
 '馆陶县',
 '魏县',
 '曲周县',
 '武安市',
 '邢台市',
 '市辖区',
 '桥东区',
 '桥西区',
 '邢台县',
 '临城县',
 '内丘县',
 '柏乡县',
 '隆尧县',
 '任县',
 '南和县',
 '宁晋县',
 '巨鹿县',
 '新河县',
 '广宗县',
 '平乡县',
 '威县',
 '清河县',
 '临西

# 生成SFT微调数据集

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [4]:
import random
import json
import time 

train_data = []

"""
Q_list 是一个包含问题模板的列表。
每个问题模板由两部分组成：
第一部分是问题的文本格式，使用占位符{}表示动态内容如城市、年份、月份、日期等）。
第二部分是指定日期格式的字符串（如 %Y-%m-%d 或 %m-%d），用于后续的日期格式化。
"""
Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    # 使用random.randint随机选择Q_list中的一个问题模板。
    Q=Q_list[random.randint(0,len(Q_list)-1)]

    # 从city_list列表中随机选择一个城市。
    city=city_list[random.randint(0,len(city_list)-1)]

    # 从1990-2025年随机选择一个年份。
    year=random.randint(1990,2025)

    # 随机选择一个月份
    month=random.randint(1,12)

    # 随机选择一天
    day=random.randint(1,28)

    # 将年、月、日组合成一个标准的日期字符串，格式为YYYY-MM-DD。
    time_str='{}-{}-{}'.format(year,month,day)

    # time.strptime将time_str转换为时间结构体。
    # time.strftime根据问题模板中的日期格式（Q[1]）重新格式化日期。
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))

    # 使用format方法将占位符替换为实际值，生成最终的问题。
    Q=Q[0].format(city=city,year=year,month=month,day=day)
    A = '城市:%s\n日期:%s'%(city, date_field)
    
    example={
        'id': 'identity_{}'.format(i),
        'conversations':[
            {
                'from':'user',
                'value':prompt_template%(Q,),
            },
            {
                'from':'assistant',
                'value':A,
            }
        ]
    }

    train_data.append(example)

with open('data/city_date_dataset.txt', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(train_data))
print('样本数量', len(train_data))

样本数量 1000


# 加载Qwen-1_8B-Chat-Int4模型

In [5]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

/home/haoyu/anaconda3/envs/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-22 19:55:15,489 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


/home/haoyu/anaconda3/envs/qwen/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/haoyu/anaconda3/envs/qwen/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/haoyu/anaconda3/envs/qwen/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
Try importing flash-attention for faster inf

# 原始模型无法按prompt要求回答json格式的城市和日期

In [6]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:很抱歉，我无法完成这个任务。因为您提供的句子中缺少了日期信息，所以我无法生成相应的JSON字符串。请提供完整的日期信息，我会很高兴帮助您完成这个任务的。

Q:青岛3-15号天气预报
A:很抱歉，我无法完成这个任务。因为您提供的句子中没有明确的日期信息，所以我无法生成相应的JSON字符串。请提供一个具体的日期或城市信息，我会很高兴帮助您完成这个任务。

Q:5月6号下雪么，城市是威海
A:这个JSON字符串应该是这样的：

```json
{
  "city": "威海",
  "date": "05/06"
}
```

这是因为句子中提到了两个信息：城市（威海）和日期（5月6日）。在Python中，你可以使用`json`模块来生成JSON字符串。以下是一个示例代码：

```python
import json

sentence = "5月6号下雪么，城市是威海"

# 将句子转换为小写并删除标点符号
sentence = sentence.lower().replace(",", "").replace(".", "").replace("吗", "").replace("的", "").replace("是", "").replace("吗", "").replace("的", "")

# 将城市和日期信息添加到JSON对象中
data = {
    "city": sentence.split("，")[0],
    "date": sentence.split("，")[1]
}

# 将JSON对象转换为字符串
json_str = json.dumps(data, indent=4)

print(json_str)
```

这段代码首先将句子转换为小写并删除了标点符号，然后将城市和日期信息添加到一个字典中。最后，它使用`json.dumps()`函数将字典转换为JSON字符串，并设置了`indent`参数以提高输出的可读性。

Q:青岛2023年12月30号有雾霾么?
A:很抱歉，我无法完成这个任务。因为您提供的句子中并没有明确的日期信息，所以我无法生成相应的JSON字符串。请提供一个具体的日期或城市信息，我会很高兴帮助您完成这个任务。

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"c

# 微调模型，生成到output_qwen

下载Qwen repo:  
git clone https://github.com/QwenLM/Qwen.git  

因为Qwen发布时间比较长了，直接使用其微调脚本会有很多版本依赖问题，所以建议新建conda env，按照Qwen requirements.txt的要求安装微调环境。  

微调命令：  
bash finetune/finetune_qlora_single_gpu.sh  -m /home/haoyu/.cache/modelscope/hub/models/qwen/Qwen-1_8B-Chat-Int4 -d data/city_date_dataset.txt

微调结束后，在Qwen目录下会生成output_qwen目录，下面是Lora adapter文件，而不是完整的模型。  

# 加载SFT后的模型

In [7]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    '/home/haoyu/work/code/DiveIntoFineTuning/Qwen/output_qwen', # 这个目录是LoRA adapter文件，而不是完整的模型文件，所以用AutoPeftModelForCausalLM
    device_map="auto",
    trust_remote_code=True
).eval()

Try importing flash-attention for faster inference...
Some weights of the model checkpoint at /home/haoyu/.cache/modelscope/hub/models/qwen/Qwen-1_8B-Chat-Int4 were not used when initializing QWenLMHeadModel: ['transformer.h.0.attn.c_proj.bias', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.0.mlp.w1.bias', 'transformer.h.0.mlp.w2.bias', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.1.mlp.w1.bias', 'transformer.h.1.mlp.w2.bias', 'transformer.h.10.attn.c_proj.bias', 'transformer.h.10.mlp.c_proj.bias', 'transformer.h.10.mlp.w1.bias', 'transformer.h.10.mlp.w2.bias', 'transformer.h.11.attn.c_proj.bias', 'transformer.h.11.mlp.c_proj.bias', 'transformer.h.11.mlp.w1.bias', 'transformer.h.11.mlp.w2.bias', 'transformer.h.12.attn.c_proj.bias', 'transformer.h.12.mlp.c_proj.bias', 'transformer.h.12.mlp.w1.bias', 'transformer.h.12.mlp.w2.bias', 'transformer.h.13.attn.c_proj.bias', 'transformer.h.13.mlp.c_proj.bias', 'transformer.h.13.mlp.w1.bias', 'transfo

# 微调后的模型可以按prompt要求回答json格式的城市和日期

In [8]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:城市:三亚
日期:2020-04-16

Q:青岛3-15号天气预报
A:城市:青岛
日期:15-03

Q:5月6号下雪么，城市是威海
A:城市:威海
日期:05-06

Q:青岛2023年12月30号有雾霾么?
A:城市:青岛
日期:2023-12-30

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:城市:北京
日期:06-01

Q:你们打算1月3号坐哪一趟航班去上海？
A:城市:上海
日期:01-03

Q:小明和小红是8月8号在上海结婚么?
A:城市:上海
日期:08-08

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:城市:东北
日期:01-15



# 模型原有问答能力仍然保留

In [9]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

青岛海边钓鱼需要注意以下几点：

1. 时间选择：青岛的天气变化较大，早晚温差大。因此，最好在早晨和傍晚的时候去钓鱼。

2. 钓鱼地点：青岛海边有很多适合钓鱼的地方，如八大关、栈桥、小鱼山等。这些地方都有丰富的鱼类资源。

3. 鱼饵选择：青岛海边的鱼类种类繁多，包括鲤鱼、鲫鱼、草鱼、鲈鱼、鳗鱼等。可以根据不同的鱼类选择合适的鱼饵。

4. 避免污染：青岛海边的水质较硬，容易造成鱼儿受伤。因此，在钓鱼时要避免直接用手接触鱼饵，也不要将鱼饵丢弃在海里。

5. 注意安全：青岛海边有海水冲刷的风险，所以在钓鱼时要注意安全，不要靠近海岸线或深水区。

6. 烹饪技巧：青岛海边的鱼类口味偏咸，烹饪时可以适当增加盐分。

以上就是青岛海边钓鱼需要注意的一些事项，希望对你有所帮助。
